<a href="https://colab.research.google.com/github/shreya-gaikwad-1/Generative-text-model/blob/main/Generative_text_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Gradio for creating a web UI and Hugging Face Transformers (latest version from GitHub)
!pip install -q gradio
!pip install -q git+https://github.com/huggingface/transformers.git

In [2]:
# Import necessary libraries
import gradio as gr # Gradio is used to create the web-based interface
import tensorflow as tf # TensorFlow is used to work with the GPT-2 model
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer  # Import GPT-2 model and tokenizer from Hugging Face


In [ ]:
# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load the pre-trained GPT-2 model with TensorFlow (TF version)
# Set the padding token to end-of-sequence token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [6]:
# Define a function that generates text from a given prompt
def generate_text(inp):
  # Encode the input prompt into token IDs (Tensor format for TensorFlow)
  input_ids = tokenizer.encode(inp, return_tensors="tf")

  # Generate output text using beam search decoding
  beam_output = model.generate(input_ids, max_length=100,        # Limit output length
                               num_beams=5,                      # Use 5 beams for better quality
                               no_repeat_ngram_size=2,           # Avoid repeating the same phrase
                               early_stopping=True)              # Stop generation when it’s likely complete

  # Decode the generated token IDs back into a readable string
  output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  # Return the text with complete sentences only
  return ".".join(output.split(".")[:-1]) + "."

In [ ]:
# Create a Gradio textbox for output
output_text = gr.Textbox()
gr.Interface(
    fn=generate_text,                                # Function to call
    inputs=gr.Textbox(label="Enter a prompt"),       # Input box for user text
    outputs=gr.Textbox(label="Generated text"),      # Output box for generated text
    title="GPT-2",                                   # Title shown on the web app
    description="OpenAI's GPT-2 can generate coherent text. Input a sentence to see how it completes it."
).launch()
